In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

#Proszę wczytać przykładowy krótki film przedstawiający obraz z kamery samochodu poruszającego się po drodze i zrealizować na nim metodę wykrywania linii poprzez transformację Hough. Wynik należy zwizualizować na sekwencji wideo.

In [ ]:
cam  = cv2.VideoCapture('/content/drive/MyDrive/AiPO/vid1.mov')
# pobranie informacji o rozmiarze klatki i ilości klatek na sekundę
width = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cam.get(cv2.CAP_PROP_FPS))

In [ ]:
# ustawienie parametrów wynikowego wideo
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
wyjscie = cv2.VideoWriter('/content/drive/MyDrive/AiPO/plik.mp4', fourcc, fps, (width, height))

while(cam.isOpened()):
    # wczytanie kolejnej klatki
    ret, frame = cam.read()
    #przerwanie pętli w przypadku zakończenia filmu
    if not ret:
      cam.release()
      break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # wykrycie krawędzi w klatce
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLines(edges, 1, np.pi/180, 200)

    # rysowanie wykrytych linii na klatce
    if lines is not None:
        for line in lines:
            rho, theta = line[0]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 1000*(-b))
            y1 = int(y0 + 1000*(a))
            x2 = int(x0 - 1000*(-b))
            y2 = int(y0 - 1000*(a))
            cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

    #Zapis klatki do pliku wyjściowego
    wyjscie.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()
wyjscie.release()

#Wczytać drugi film, przedstawiający nagranie samochodów na autostradzie. Przenieść do odcieni szarości. Stworzyć sekwencję różnic pomiędzy kolejnymi klatkami. Utworzone różnice wykorzystać jako maski na oryginalnych klatkach.

In [ ]:
cap  = cv2.VideoCapture('/content/drive/MyDrive/AiPO/vid2.mov')
# pobranie informacji o rozmiarze klatki i ilości klatek na sekundę
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
frame_size = (width, height)

out = cv2.VideoWriter('/content/drive/MyDrive/AiPO/plik2.avi', fourcc, fps, frame_size)

while(cap.isOpened()):
    # wczytanie kolejnej klatki
    ret, frame = cap.read()

    #przerwanie pętli w przypadku zakończenia filmu
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    gray = cv2.resize(gray, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

    # wygenerowanie maski na podstawie różnicy między kolejnymi klatkami
    if 'prev_frame' in locals():
        diff = cv2.absdiff(gray, prev_frame)
        _, mask = cv2.threshold(diff, 100, 255, cv2.THRESH_BINARY+ cv2.THRESH_OTSU)
        mask = cv2.dilate(mask, None, iterations=2)
        mask = cv2.resize(mask, frame_size[::-1])
    else:
        mask = np.zeros_like(gray)

    #nałożenie maski na oryginalną klatkę
    masked_frame = cv2.bitwise_and(frame, frame, mask=cv2.resize(mask, (frame.shape[1], frame.shape[0])))

    out.write(masked_frame)

    # aktualizacja poprzedniej klatki
    prev_frame = gray

cap.release()
out.release()
cv2.destroyAllWindows()